### Exploratory Analysis of Covid-19 Cases, Hospitalizations, and Hospital Capacity

#### Aaron McAdie

I've been really curious about how the current case load in the US compares with regional hospital beds, and which regions are approaching their capacity.  The folks at the New York times just open sourced their [county-level Covid-19 case dataset](https://github.com/nytimes/covid-19-data), and it provides a great jumping off point to probe the question.  It's also a good opportunity to brush up on Python, since I do ~90% of my day job using R.

Also, since I started working on this, I came across the [incredible modeling and data viz](https://covid19.healthdata.org/projections) done by the UW IHME team.  They have done a way better job than I have the subject matter expertise to match, so I'll pull in their projections and explore it from a few different angles.

In [ ]:
import requests
import re
import zipfile
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt

#### Dataset 1 - NYTimes Cases, thanks NYT!

In [ ]:
cases = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [ ]:
cases.head()

In [ ]:
cases.dtypes

In [ ]:
cases = cases.assign(
    date = pd.to_datetime(cases['date']),
    fips = cases['fips'].astype(pd.Int32Dtype())
)

Cases are cumulative, we want new cases each day to estimate the hospital case load

In [ ]:
cases['cases_shifted'] = (
    cases.groupby(['county', 'state'])
    .cases
    .shift(1)
    .fillna(0)
    .astype(int)
)

In [ ]:
cases['cases_new'] = cases['cases'] - cases['cases_shifted']

Check logic by plotting King County data.  I've been wanting to see more presentations of new cases per day rather than the cumulative plots that are more commonly displayed to get a better feel for how we are 'flattening the curve'.

In [ ]:
king = cases.query('county == "King" & state == "Washington"')

In [ ]:
base = alt.Chart(king[['date', 'cases', 'cases_new']]).encode(alt.X('monthdate(date):O', title = 'Date'))
bars = base.mark_bar(color = '#65799b').encode(y = 'cases_new', tooltip = 'cases_new')
bars.properties(title = 'King County WA Epidemiological Curve (new cases per day)')

In [ ]:
line = base.mark_line(color = '#e23e57').encode(y = 'cases', tooltip = 'cases')

(bars + line).properties(title = 'King County Cumulative and Daily Covid-19 Cases')

Passes sanity check.  Interesting that it looks like there are 4 roughly linear regimes on the cumulative curve, just from eyeballing

In [ ]:
statewide = (
    cases.groupby(['date', 'state'])
    .aggregate({'cases':'sum'})
    .reset_index()
)

In [ ]:
statewide.head()

#### State by State Cases Over Time

In [ ]:
(alt.Chart(statewide)
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q')
 .properties(title = 'Cumulative US Covid-19 Cases by Day', width = 600)
)

In [ ]:
(alt.Chart(statewide[statewide['state'] != 'New York'])
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q',
         tooltip = ['date', 'state', 'cases'])
 .properties(title = 'Cumulative US Covid-19 Cases by Day (NY removed)', width = 600)
)

### Dataset 2 - Population and Demographics by County (U.S. Census Bureau)

In [ ]:
pop = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/counties/asrh/cc-est2017-alldata.csv',
                 encoding='ISO-8859-1')

In [ ]:
pop.head()

#### Join cases and census data to get per-capita infections

In [ ]:
# Filter to get most recent census data (2017)
# strip the ' County' from the census data to join to cases
# start with total age

pop17 = (pop
       .query('YEAR == 10 and AGEGRP == 0')
       .assign(county_key = lambda x: x['CTYNAME'].str.extract(r'([A-Za-z\s\.\-]*)(?=\sCounty)'))
       .loc[:,['STNAME', 'CTYNAME', 'county_key', 'TOT_POP']]
      )

In [ ]:
# Manually fix a few County/district names so they join to the NYT cases dataset

# All cases for the five boroughs of New York City 
# (New York, Kings, Queens, Bronx and Richmond counties) 
# are assigned to a single area called New York City.
nyc_idx = (pop17.CTYNAME.str.contains(r'New York|Kings|Queens|Bronx|Richmond')) \
    & (pop17.STNAME == 'New York') # and vs & is so confusing to me
    
pop17.loc[nyc_idx, 'county_key'] = 'New York City'

# D.C
pop17.loc[pop17.CTYNAME == 'District of Columbia', 'county_key'] = 'District of Columbia'

In [ ]:
pop17.loc[nyc_idx,]

In [ ]:
pop17_clean = (pop17
               .groupby(['county_key', 'STNAME'])
               .agg({'TOT_POP':'sum'})
               .reset_index()
              )

In [ ]:
cases_percap = cases.merge(pop17_clean, how = 'left', left_on = ['county', 'state'], right_on = ['county_key', 'STNAME'])

In [ ]:
cases_percap.head()

In [ ]:
# check mismatches
sum(pd.isnull(cases_percap.TOT_POP))

In [ ]:
cases_percap.loc[pd.isnull(cases_percap.TOT_POP), 'county'].value_counts()

Still some mismatches to iron out, but let's go forward with this data for now

In [ ]:
cases_percap['cases_per10k'] = cases_percap.cases / cases_percap.TOT_POP * 10000
cases_percap['cases_new_per1k'] = cases_percap.cases_new / cases_percap.TOT_POP * 1000
cases_percap['county_label'] = cases_percap.county + ', ' + cases_percap.STNAME

In [ ]:
cases_percap.head()

In [ ]:
# cumulative cases for counties with > 1 million people
# save data as JSON for better performance with Altair

cases_percap_url = 'cases_percap.json'

cases_percap[cases_percap.TOT_POP > 1000000].to_json(cases_percap_url, orient = 'records')

# can't figure out why the file url isn't working, can read it in just fine with open('cases_percap.json')
# these data are small enough for just saving the dataset with the chart though
cases_percap_big = cases_percap.loc[cases_percap.TOT_POP > 1000000, 
                                    ['date', 'cases', 'cases_per10k', 'county_label', 'cases_new_per1k', 'STNAME', 'TOT_POP']]

(alt.Chart(cases_percap_big)
    .mark_line(color = 'grey', opacity = 0.6)
    .encode(x = 'date:T',
            y = 'cases_per10k:Q',
            color = 'county_label:N',
            tooltip = ['county_label:N', 'cases:Q'])
    .properties(height = 400, 
                width = 700,
               title = 'Per Capita Covid-19 Cases, Counties > 1M')
    .interactive())

Getting somewhere, but need a way to visually distinguish. Maybe a panel of plots with cross-filtering?

In [ ]:
(alt.Chart(cases_percap_big)
    .mark_point()
    .encode(x = 'TOT_POP:Q',
            y = 'STNAME:N',
            tooltip = 'county_label'))

In [ ]:
# combine the dot plot and the cumulative case plot, with cross-filtering
brush = alt.selection(type = 'interval', resolve = 'global')
base = alt.Chart(cases_percap_big)

pop_points = (base.mark_point()
             .encode(x = alt.X('TOT_POP:Q', axis = alt.Axis(title = 'Population')),
                     y = alt.Y('STNAME:N', axis = alt.Axis(title = 'State')))
             .add_selection(brush)
             .properties(height = 400, 
                         width = 200,
                         title = 'Counties > 1M (brush to select)'))

percap_cases_line = (base.mark_line(color = 'grey', opacity = 0.6)
            .encode(x = alt.X('date:T', axis = alt.Axis(title = 'Date')),
                    y = alt.Y('cases_per10k:Q', axis = alt.Axis(title = 'Cases per 10,000 people')),
                    color = 'county_label:N',
                    tooltip = 'county_label:N')
            .properties(height = 400, 
                        width = 450,
                        title = 'Per Capita Covid-19 Cases, Counties > 1M')
            .transform_filter(brush))

(pop_points | percap_cases_line).save('percap_covid_big_counties.html')

pop_points | percap_cases_line

Wow, Altair is rad!

Stealing idea that Nate Cohn had for new cases vs prevalence, updating with new data
tweaking by using a moving average of new cases per capita rather than % change in cumulative
cases, since we are well into the pandemic, the same absolute number of new cases will be smaller and
smaller as a percent of the cumulative total

In [ ]:
# create column for rolling 5 day average of new cases
cases_percap_big = cases_percap_big.sort_values(['county_label', 'date'])
#cases_percap_big[~cases_percap_big['cases_new_per1k'].isna()]

cases_percap_big['cases_new_per1k_MA'] = (cases_percap_big
                                          .groupby('county_label')['cases_new_per1k']
                                          .transform(lambda x: x.rolling(5).mean()))

In [ ]:
lines = (alt.Chart(cases_percap_big)
    .mark_line()
    .encode(x = 'cases_per10k:Q',
            y = 'cases_new_per1k_MA:Q',
            stroke = 'county_label:N',
            tooltip = 'county_label:N')
    .interactive()
    .properties(height = 400, width = 700))

# get latest record per county for label
cases_percap_big['latest'] = cases_percap_big.groupby('county_label')['date'].transform('max')
cases_percap_big_latest = cases_percap_big[cases_percap_big.date == cases_percap_big.latest]

labels = (alt.Chart(cases_percap_big_latest)
          .mark_point()
          .encode(x = 'cases_per10k:Q',
                  y = 'cases_new_per1k_MA:Q')
          .mark_text(align = 'left',
                    baseline = 'middle',
                    color = 'grey')
          .encode(text = 'county_label:N'))

lines + labels

In [ ]:
#oh snap, do the side-by side panel, with the scatter being most recent new cases per 1k, and the x being population (or 
#maybe log population), and then the right plot the new cases vs cases per 10k

# add a max cumulative cases column for the scatter plot, could do with the Altair call, but hopefully this will get the
# selection behavior I'm looking for

max_cases = (cases_percap_big
             .groupby('county_label')
             .agg({'cases_per10k': 'max'})
             .reset_index()
             .rename(columns = {'cases_per10k': 'max_cases_per10k'})
            )

cases_percap_big = cases_percap_big.merge(max_cases, how = 'inner', on = 'county_label')


brush2 = alt.selection(type = 'interval', 
                       empty = 'all')

scatter_pop = (alt.Chart(cases_percap_big)
               .mark_point()
               .encode(x = alt.X('TOT_POP:Q', 
                                 axis = alt.Axis(title = 'County Population (log2)'),
                                 scale = alt.Scale(type = 'log', base = 2)),
                       y = alt.Y('max_cases_per10k:Q',
                                axis = alt.Axis(title = 'Total Cases per 10,000 (log2)'),
                                scale = alt.Scale(type = 'log', base = 2)))
               .add_selection(brush2)
               .properties(height = 400, 
                           width = 200,
                           title = ['brush to select population range', 
                                    'drag selection to compare similar size counties'])
              )

spread_chart = (alt.Chart(cases_percap_big)
                .mark_line()
                .encode(x = alt.X('cases_per10k:Q',
                                  axis = alt.Axis(title = 'Cumulative Cases per 10,000')),
                        y = alt.Y('cases_new_per1k_MA:Q',
                                  axis = alt.Axis(title = '5-day Average of New Cases per 1,000')),
                        color = 'county_label:N',
                        tooltip = 'county_label:N')
                .transform_filter(brush2)
                .properties(height = 400, 
                            width = 450,
                            title = 'Transmittance vs Prevalence, Counties > 1M')
                .interactive()
               )

scatter_pop | spread_chart

Can't figure out how to add the labels on the spread chart but pretty happy with this!

The biggest things that jumped out are that Miami-Dade and Cook Illinois are the two big counties that could be the next major oubreaks.  The virus is relatively prevalent in those communities, and the rate of new cases is also relatively high on a per-capita basis.

Of the smaller counties (note that this is relative, all counties plotted had more than 1 million residents in 2017), by far the most prevalence/fasted spread is in Suffolk and Nassau counties, which makes sense since these are adjacent to NYC.  Middlesex MA and Phildelphia PA look to be on a trajectory of rapid spread.  Wayne MI (which includes Detroit) has had quite a dramatic decrease in new cases over the last few days (provided they are still testing at the same rate).

### Dataset 3 - Hospitalization Rates by Age Group (CDC)

The CDC published a table of hospitalization, ICU, and death rates by age group, based on ~2500 hospitalizations in the US in Feb/Mar.  This could be used to get a rough estimate of particular types of beds needed, namely pediatric, which is of high interest for us here at Seattle Children's.

In [ ]:
# get html from CDC
cdc_html = requests.get('https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm')
soup = BeautifulSoup(cdc_html.content)

In [ ]:
# find table
tables = soup.find_all('table')
tables[0].find_all('tbody')

In [ ]:
table = tables[0].find('tbody')

rows_dict = dict()

for idx, row in enumerate(table.find_all('tr')):
    temp_list = list()
    for col in row.find_all('td'):
        temp_list.append(col.string)
    rows_dict[idx] = temp_list

In [ ]:
cdc_df = pd.DataFrame(rows_dict).T
cdc_df.columns = ['age_range', 'hosp_rate', 'icu_rate', 'death_rate']
cdc_df

In [ ]:
# clean up a bit, extract relevant info into usable columns
# self-challenge by doing this with vectorized pandas functions rather than standard regex

cdc_df = (cdc_df
          .assign(n = lambda x: x['age_range'].str.extract(r'\(([0-9,]{3,})\)'),
                  age_range = lambda x: x['age_range'].str.extract(r'(.*)\s\([0-9,]{3,}\)'),
                  hosp_lo = lambda x: x['hosp_rate'].str.extract(r'([0-9\.]{2,}).[0-9\.]{2,}'),
                  hosp_hi = lambda x: x['hosp_rate'].str.extract(r'[0-9\.]{2,}.([0-9\.]{2,})'))
         )

cdc_df

In [ ]:
# strip off total into separate df
cdc_total = cdc_df[cdc_df.age_range.str.contains('Total')]
cdc_df = cdc_df[~cdc_df.age_range.str.contains('Total')]

### Dataset 4 - Hospital Locations and Capacity (U.S. Homeland Infrastructure Foundation-Level Data)

In [ ]:
hosp_req = requests.get('https://opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0.geojson')

In [ ]:
hosp_json = hosp_req.json()
hosp_json['features'][0]

hosp_data = [x['properties'] for x in hosp_json['features']]

hosp_df = pd.DataFrame(hosp_data)

hosp_df.head()